In [1]:
import numpy as np
from math import isnan
import itertools
import pprint
pp = pprint.PrettyPrinter(indent=4)
fpath = '/checkpoint/massran/async_maopy_playground/slurm/qp/n{world_size}/{tag_key}{tag_val}/{graph}/{alg}/r{rank}_n{world_size}.npz'

In [157]:
print('Initializing plotting')
print('------------------------------------------')

# Whether to generate plots for paper
debug = False

if debug:
    from bokeh.io import output_notebook, show, export_svgs
    from bokeh.plotting import figure
    output_notebook()
else:
    import seaborn as sns
    import matplotlib.pyplot as plt
    sns.set()
    sns.set_style('whitegrid')
    figsize_single = (6, 5)
    figsize_double = (12.5, 5)
    fontsize = 15
    linestyles = ['-', '--']
    linewidth = 3.0
    colors = ['#1f77b4',  # blue
               '#2ca02c',  # green
               '#d62728',  # red
               '#8c564b',  # brown
               '#e377c2',  # pink
               '#7f7f7f',  # gray
               '#bcbd22',  # yellow
               '#17becf']  # turquoise
    
image_dir = './assets/'
legend_alg = {
    'agp': 'AGP',
    'gp': 'SGP',
    'pd': 'PD',
    'ep': 'EP',
    'asy-sonata': 'Asy-SONATA'
}
print('Done.')

Initializing plotting
------------------------------------------
Done.


In [3]:
def load_data(rank=0, world_size=2, alg='gp', tag=None, graph='erdos-renyi'):
    """ Returns lits of flattened arguments and wallclock-times """
    arg_list, time_list = [], []
    data = np.load(fpath.format(world_size=world_size, graph=graph,
                                tag_key=list(tag.keys())[0], tag_val=list(tag.values())[0],
                                alg=alg, rank=rank), allow_pickle=True)['argmin_est'].item()
    if alg == 'asy-sonata':
        data = data.history
    for itr in data:
        t, v = data[itr]
        arg_list.append(v.flatten())
        time_list.append(t)
    return time_list, arg_list

def compute_global_cond(fpath='qp_data_sg.npz'):
    """ Returns condition number of objective """
    print('Loading data to compute condition number...')
    data_fpath = './datasets/' + fpath
    data = np.load(data_fpath)
    a_m = data['A']
    print('Computing condition number...')
    kai = np.linalg.cond(a_m.T.dot(a_m))
    return kai

def compute_local_conds(world_size=2, fpath='qp_data_sg.npz'):
    """ Returns condition number of objective """
    
    print('Loading data to compute condition numbers...')
    data_fpath = './datasets/' + fpath
    data = np.load(data_fpath)
    a_m = data['A']
    b_v = data['b']
    split_size = int(a_m.shape[0] // world_size)
    cond_list = []

    for rank in range(world_size):
        start_index = int(rank * split_size)
        if rank < (world_size - 1):
            end_index = int((rank+1) * split_size)
            r_a_m = a_m[start_index:end_index, :]
        else:
            r_a_m = a_m[start_index:, :]

        print('Computing condition number for rank (%s)...' % rank)
        kai = np.linalg.cond(r_a_m.T.dot(r_a_m))
        cond_list.append(kai)

    return cond_list

def compute_local_xstar_list(world_size=2, fpath='qp_data_sg.npz'):
    """ Returns xstar for individual agent's subset of data"""
    
    print('Loading local data...')
    data_fpath = './datasets/' + fpath
    data = np.load(data_fpath)
    a_m = data['A']
    b_v = data['b']
    split_size = int(a_m.shape[0] // world_size)
    xstar_list = []
    for rank in range(world_size):
        start_index = int(rank * split_size)
        if rank < (world_size - 1):
            end_index = int((rank+1) * split_size)
            r_a_m = a_m[start_index:end_index, :]
            r_b_v = b_v[start_index:end_index]
        else:
            r_a_m = a_m[start_index:, :]
            r_b_v = b_v[start_index:]

        print('Computing local minimzer for rank (%s)...' % rank)
        xstar = np.linalg.inv(r_a_m.T.dot(r_a_m)).dot(r_a_m.T.dot(r_b_v))
        xstar_list.append(xstar)

    return xstar_list

def load_global_objective(fpath='qp_data_sg.npz'):
    """ Returns lambda optimality measure"""
    print('Loading optimality measure...')
    data_fpath = './datasets/' + fpath
    data = np.load(data_fpath)
    a_m = data['A']
    b_v = data['b']
    xstar = data['x_star']
    print('\t Loaded data used to construct measure...', a_m.shape, b_v.shape)
    return lambda x: np.linalg.norm(x - xstar)

def get_world_itr_data(objective, world_size=2, alg='gp', tag=None, graph='erdos-renyi', eval_at_agg=True):
    """ Returns (undistilled) x/y data for algorithm/world_size/rank """

    print('Loading all itr-world data (%s) for alg (%s)...' % (world_size, alg))
    num_updates_list = []
    time_list, arg_list = [], []
    for rank in range(world_size):
        tl, al = load_data(rank=rank, world_size=world_size, alg=alg, tag=tag, graph=graph)
        if not eval_at_agg:
            al = list(map(objective, al))
        num_updates_list.append(len(tl))
        time_list.append(tl)
        arg_list.append(al)

    print('Aggregating data...')
    def aggregate(xrank_list):
        """ Aggregate cross-rank list-data across ranks """
        agg_list = []
        for xrank_arg in zip(*xrank_list):
            if eval_at_agg:
                temp = sum(xrank_arg) / len(xrank_arg)
            else:
                temp = min(xrank_arg)
            agg_list.append(temp)
        return np.array(agg_list)
    arg_list = aggregate(arg_list)

    if eval_at_agg:
        print('Computing optimlaity...')
        arg_list = np.array(list(map(objective, arg_list)))

    return arg_list, num_updates_list

def get_world_time_data(objective, world_size=2, alg='gp', tag=None, graph='erdos-renyi', indx=100, step=1.,
                        distill=True, eval_at_agg=True):
    """ Returns (distilled) x/y data for algorithm/world-size """

    print('Loading all time-world data (%s) for alg (%s)...' % (world_size, alg))
    time_list, arg_list = [], []
    for rank in range(world_size):
        tl, al = load_data(rank=rank, world_size=world_size, alg=alg, graph=graph, tag=tag)
        if not eval_at_agg:
            al = list(map(objective, al))
        time_list.append(tl)
        arg_list.append(al)

    def distill_data(t_rank, a_rank):
        """ Keep subset of data to enable async-aggregation across ranks """
        keep_indices = []
        for t_max in x_time:
            # For each t_max, find the single time-index to keep
            i_keep, t_prev = 0, 0
            for i, t in enumerate(t_rank, i_keep):
                if t > t_max:
                    break
                if t_prev <= t:
                    i_keep = i
            keep_indices.append(i_keep)
        temp = list(np.array(a_rank)[keep_indices])
        return temp

    def aggregate(xrank_list):
        """ Aggregate cross-rank list-data across ranks """
        agg_list = []
        for xrank_arg in zip(*xrank_list):
            if eval_at_agg:
                temp = sum(xrank_arg) / len(xrank_arg)
            else:
                temp = min(xrank_arg)
            agg_list.append(temp)
        return np.array(agg_list)

    if distill:
        print('Distilling data...')
        x_time = [t * step for t in range(indx)]
        arg_time = []
        for tl, al in zip(time_list, arg_list):
            temp = distill_data(tl, al)
            arg_time.append(temp)
    else:
        arg_time = arg_list
        x_time = aggregate(time_list)

    print('Aggregating data...')
    arg_time = aggregate(arg_time)

    if eval_at_agg:
        print('Computing optimlaity...')
        arg_time = np.array(list(map(objective, arg_time)))

    return x_time, arg_time

def load_config_data(alg_list, world_size_list, tag_list, graph_list, t_indx=2000,
                     t_step=0.1, eval_at_agg=True, metric=None):
    if metric is None:
        metric = globjective

    data_dict = {}
    for alg in alg_list:
        data_dict[alg] = {}
        for ws in world_size_list:
            data_dict[alg][ws] = {}
            for gph in graph_list:
                data_dict[alg][ws][gph] = {}
                data_dict[alg][ws][gph]['time'] = {}
                data_dict[alg][ws][gph]['obj_time'] = {}
                data_dict[alg][ws][gph]['obj_itr'] = {}
                data_dict[alg][ws][gph]['num_updates'] = {}

                for tag in tag_list:
                    # Create dictionary key
                    tag_k, tag_v = list(tag.keys())[0], list(tag.values())[0]
                    tag_str = '%s:%s' % (tag_k, tag_v)
                    try:
                        # Load global time data
                        tl, yl = get_world_time_data(globjective, ws, alg, tag, gph, indx=t_indx,
                                                     step=t_step, eval_at_agg=eval_at_agg)
                        data_dict[alg][ws][gph]['time'][tag_str] = tl
                        data_dict[alg][ws][gph]['obj_time'][tag_str] = yl
                        # Load global iteration data
                        yl, ul = get_world_itr_data(globjective, ws, alg, tag, gph, eval_at_agg=eval_at_agg)
                        data_dict[alg][ws][gph]['obj_itr'][tag_str] = yl
                        data_dict[alg][ws][gph]['num_updates'][tag_str] = ul
                    except Exception as e:
                        print(e)
                        data_dict[alg][ws][gph]['time'][tag_str] = None
                        data_dict[alg][ws][gph]['obj_time'][tag_str] = None
                        # Load global iteration data
                        data_dict[alg][ws][gph]['obj_itr'][tag_str] = None
                        data_dict[alg][ws][gph]['num_updates'][tag_str] = None
    return data_dict

In [4]:
globjective = load_global_objective()
print('Done.')

Loading optimality measure...
	 Loaded data used to construct measure... (2560000, 50) (2560000,)
Done.


In [ ]:
local_kais = compute_local_conds(world_size=40)
pp.pprint(local_kais)

global_kai = compute_global_cond()
pp.pprint(global_kai)

In [ ]:
print('Plotting Condition Numbers')
print('------------------------------------------')

num_colors = 40
if debug:
    from bokeh.palettes import Dark2_8 as palette
    colors = itertools.cycle(palette)
    g_colors = (c[1] for c in zip(range(num_colors), colors))
    p = figure(plot_width=800, plot_height=400,
               title='Local Objective Condition Numbers',
               y_axis_type='linear')
    p.yaxis.axis_label = 'condition number'
    p.xaxis.axis_label = 'rank'
else:
    g_colors = itertools.cycle(colors)
    g_colors = (c[1] for c in zip(range(num_colors), g_colors))
    _, ax = plt.subplots(nrows=1, ncols=1, figsize=figsize_single)

x = np.arange(len(local_kais))
y = [global_kai for _ in range(len(local_kais))]

if debug:
    p.line(x, y, color='red', line_width=3, legend_label='global condition number')
    p.scatter(x, local_kais, radius=.5,
              fill_color='blue', fill_alpha=0.6,
              line_color=None, legend_label='Local condition number')
else:
    ax.plot(x, y, linestyle=linestyles[1], linewidth=1.*linewidth,
            label=r'Global condition number')
    ax.scatter(x, local_kais, marker='o', legend_label='Local condition number')

if debug:
    p.legend.location = 'top_left'
    p.output_backend = 'svg'
    show(p)
else:
    ax.set_title('Condition Numbers of Local Objectives', fontsize=fontsize)
    ax.set_xlabel(r'Rank', fontsize=fontsize)
    ax.tick_params(axis='both', labelsize=fontsize)
    ax.set_ylabel(r'Condition number', fontsize=fontsize)
    ax.legend(loc='upper left', fontsize=fontsize)
    fig_newton = './assets/condition_numbers.pdf'
    plt.savefig(fig_newton, bbox_inches='tight', pad_inches=0)
    plt.show()

print('Done.')

In [ ]:
print('Loading and processing experiment data')
print('------------------------------------------')
eval_at_agg = True
alg_list = ['agp']
world_size_list = [40]
tag_list = [
    {'tauproc':1},
    {'tauproc':4},
    {'tauproc':8},
    {'tauproc':16},
    {'tauproc':32}
]
t_indx, t_step = 2500, .2
print('Data-Tag: %s' % list(tag_list[0].keys())[0])

tau_data_dict = load_config_data(alg_list, world_size_list, tag_list, t_indx, t_step, eval_at_agg)

print('Done.')

In [ ]:
print('Computing theoretical re-weighting values')
print('------------------------------------------')

tau_list = [list(t.values())[0] for t in tag_list]
reweighting_list = [None for _ in range(len(tau_list))]

alg, world_size = alg_list[0], world_size_list[0]
for i, key in enumerate(tau_data_dict[alg][world_size]['num_updates']):
    ul = tau_data_dict[alg][world_size]['num_updates'][key]
    nrm = sum(ul)
    reweighting_list[i] = [n / nrm for n in ul]

print('Computed re-weighting values...')
pp.pprint(reweighting_list)

print('Done.')

In [ ]:
print('world-size %s' % world_size)
xstar_list = compute_local_xstar_list(world_size)
print('Done.')

In [ ]:
print('Computing suboptimality of local minimizers and theoretical bound')
print('------------------------------------------')

# Metric is l2-distance to global minimizer
global_xstar = 0.
for rw, xs in zip(reweighting_list[0], xstar_list):
    global_xstar += rw * xs
metric = lambda x: np.linalg.norm(x - global_xstar)

print('Computing suboptimality of re-weighted optimizer')
subopt_list = []
reweighted_xstar_list = []
for t in range(len(tau_list)):
    reweighted_xstar = 0.
    for rw, xs in zip(reweighting_list[t], xstar_list):
        reweighted_xstar += rw * xs
    reweighted_xstar_list.append(reweighted_xstar)
    subopt = metric(reweighted_xstar)
    subopt_list.append(subopt)
print('sub-optimalities:')
pp.pprint(subopt_list)

print('Computing Theorem-5 bounds for various tau')
tau_bounds = []
for t in range(len(tau_list)):
    dfu = sum([abs(rw - (1./world_size)) for rw in reweighting_list[t]]) / world_size
    print(dfu)
    sbar = max([metric(xs_i) for xs_i in xstar_list])
    tbound = sbar * (dfu)**.5 * (global_kai)**.5 / 2**.5
    tau_bounds.append(tbound)
print('tau-bounds:')
pp.pprint(tau_bounds)

print('Done.')

In [ ]:
print('Plotting Suboptimality vs. delay')
print('------------------------------------------')

num_colors = 40
if debug:
    from bokeh.palettes import Dark2_8 as palette
    colors = itertools.cycle(palette)
    g_colors = (c[1] for c in zip(range(num_colors), colors))
    p = figure(plot_width=800, plot_height=400,
               title='Suboptimality of Reweighted Minimizer',
               y_axis_type='linear')
    p.yaxis.axis_label = 'subpotimality'
    p.xaxis.axis_label = 'tauratemax'
else:
    g_colors = itertools.cycle(colors)
    g_colors = (c[1] for c in zip(range(num_colors), g_colors))
    _, ax = plt.subplots(nrows=1, ncols=1, figsize=figsize_single)

# distill data
k_indices = [0,1,2,3,4]
k_tau_list = np.array(tau_list)[k_indices]
k_subopt_list = np.array(subopt_list)[k_indices]
k_tau_bound_list = np.array(tau_bounds)[k_indices]

if debug:
    # plot theoretical bound
    p.line(k_tau_list, k_tau_bound_list, color='red', legend='Theoretical Bound')
    p.circle(k_tau_list, k_tau_bound_list, color='red', size=12, legend='Theoretical Bound')
    # plot actual distances
    p.line(k_tau_list, k_subopt_list, legend='Computed Distance')
    p.circle(k_tau_list, k_subopt_list, size=12, legend='Computed Distance')
else:
    color = next(g_colors)
    # plot theoretical bound
    ax.plot(k_tau_list, k_tau_bound_list, color=None, linestyle=linestyles[1], linewidth=1.*linewidth,
            label='Thm.[3]', marker='s', markersize=12)
    color = next(g_colors)
    # plot actual distances
    ax.plot(k_tau_list, k_subopt_list, color=None, linestyle=linestyles[1], linewidth=1.*linewidth,
            label=r'$\Vert \mathbf{x^{\star}_K} - \mathbf{\overline{x}^{\star}} \Vert_2$',
            marker='s', markersize=12)

if debug:
    p.legend.location = 'bottom_right'
    p.output_backend = 'svg'
    show(p)
else:
    ax.set_title('Suboptimality of Re-weighted Minimizer', fontsize=fontsize)
    ax.set_xlabel(r'$\overline{\tau}^{(proc)}$', fontsize=fontsize)
    ax.tick_params(axis='both', labelsize=fontsize)
    ax.set_ylabel(r'$\Vert \mathbf{x^{\star}_K} - \mathbf{x^{\star}} \Vert_2$', fontsize=fontsize)
    ax.legend(loc='upper left', fontsize=fontsize)
    fig_newton = './assets/x_K_sweep_tau.pdf'
    plt.savefig(fig_newton, bbox_inches='tight', pad_inches=0)
    plt.show()

print('Done.')

In [ ]:
print('Plotting tau-sweep error vs. iterations')
print('------------------------------------------')

num_colors = 50
if debug:
    from bokeh.palettes import Dark2_8 as palette
    colors = itertools.cycle(palette)
    g_colors = (c[1] for c in zip(range(num_colors), colors))
    p = figure(plot_width=800, plot_height=400,
               title='Average error sequence',
               y_axis_type='log')
    p.yaxis.axis_label = 'global loss'
    p.xaxis.axis_label = 'iterations'
    p.output_backend = 'svg'
else:
    g_colors = itertools.cycle(colors)
    g_colors = (c[1] for c in zip(range(num_colors), g_colors))
    _, axes = plt.subplots(nrows=1, ncols=2, figsize=figsize_double, sharey=False)

bound_indx = -1
if debug:
    xl = np.arange(1100)
    tbl = [tau_bounds[bound_indx] for _ in xl]
    sbl = [subopt_list[bound_indx] for _ in xl]
    p.line(xl, tbl, color='red', line_width=5, legend='Theoretical Bound-(tauproc:32)')
    p.line(xl, sbl, color='blue', line_width=5, legend='Suboptimality of xstar_K-(tauproc:32)')
else:
    xl_all = [np.arange(1100), np.arange(550)]
    tbl_all = [[tau_bounds[bound_indx] for _ in xl_all[0]],
               [tau_bounds[bound_indx] for _ in xl_all[1]]]
    sbl_all = [[subopt_list[bound_indx] for _ in xl_all[0]],
               [subopt_list[bound_indx] for _ in xl_all[1]]]
    cl_all = [next(g_colors), next(g_colors)]
    for ax, xl, tbl, sbl in zip(axes, xl_all, tbl_all, sbl_all):
        ax.semilogy(xl, tbl, color=None, linestyle=linestyles[1], linewidth=1.*linewidth,
                    label=r'$(\overline{\tau}^{(proc)}:32)$ Thm.[3]')
        color = next(g_colors)
        ax.semilogy(xl, sbl, color=None, linestyle=linestyles[1],
                    label=r'$(\overline{\tau}^{(proc)}:32)\ \Vert \mathbf{\overline{x}^{\star}_K} - \mathbf{\overline{x}^{\star}} \Vert_2$',
                    linewidth=1.*linewidth)

# metric = lambda x: np.linalg.norm(x - reweighted_xstar_list[-1])
# tau_data_dict = load_config_data(alg_list, world_size_list, [tag_list[-1]], t_indx, t_step, eval_at_agg, metric=metric)


for alg in tau_data_dict:
    for ws in tau_data_dict[alg]:
        for key in tau_data_dict[alg][ws]['obj_itr']:
            color = next(g_colors)
            if debug:
                yl = tau_data_dict[alg][ws]['obj_itr'][key]
                xl = np.arange(len(yl))
                legend = '%s-(n:%s, %s) ' % (legend_alg[alg], ws, key)
                p.line(xl, yl, legend=legend, color=color, line_width=2)
            else:
                legend = r'$(\overline{\tau}^{(proc)}:$%s)' % (key.split(':')[-1])
                yl_itr = tau_data_dict[alg][ws]['obj_itr'][key]
                xl_itr = np.arange(len(yl_itr))        
                axes[0].semilogy(xl_itr, yl_itr, color=None, linestyle=linestyles[0],
                                 linewidth=linewidth, label=legend)
                yl_time = tau_data_dict[alg][ws]['obj_time'][key]
                xl_time = tau_data_dict[alg][ws]['time'][key]
                axes[1].semilogy(xl_time, yl_time, color=None, linestyle=linestyles[0], 
                                 linewidth=linewidth, label=legend)

if debug:
    show(p)
else:
    for ax in axes:
        ax.set_title('Training Error', fontsize=fontsize)
        ax.tick_params(axis='both', labelsize=fontsize)
        ax.legend(loc='upper right', fontsize=fontsize)
    axes[0].set_ylabel(r'$\Vert \mathbf{\overline{x}}[k] - \mathbf{x^{\star}} \Vert_2$', fontsize=fontsize)
    axes[0].set_xlabel(r'Iterations $k$', fontsize=fontsize)
    axes[1].set_xlabel(r'Time $t[k]$ (seconds)', fontsize=fontsize)
    fig_newton = './assets/agp_train_tau-lines.pdf'
    plt.savefig(fig_newton, bbox_inches='tight', pad_inches=0)
    plt.show()

print('Done.')

In [5]:
print('Loading and processing experiment data')
print('------------------------------------------')
eval_at_agg = True

alg_list = ['asy-sonata', 'agp', 'gp', 'pd', 'ep']
world_size_list = [40]
tag_list = [
    {'step-size':0.001},
    {'step-size':0.01},
    {'step-size':0.1},
    {'step-size':1.0},
    {'step-size':10.0},
    {'step-size':100.0},
]
graph_list = ['erdos-renyi', 'ring']
t_indx, t_step = 1600, 0.75
print('Data-Tag: %s' % list(tag_list[0].keys())[0])

lr_data_dict = load_config_data(alg_list, world_size_list, tag_list, graph_list, t_indx, t_step, eval_at_agg)

print('Done.')

Loading and processing experiment data
------------------------------------------
Data-Tag: step-size
Loading all time-world data (40) for alg (asy-sonata)...
Distilling data...
Aggregating data...
Computing optimlaity...
Loading all itr-world data (40) for alg (asy-sonata)...
Aggregating data...
Computing optimlaity...
Loading all time-world data (40) for alg (asy-sonata)...
Distilling data...
Aggregating data...
Computing optimlaity...
Loading all itr-world data (40) for alg (asy-sonata)...
Aggregating data...
Computing optimlaity...
Loading all time-world data (40) for alg (asy-sonata)...
Distilling data...
Aggregating data...
Computing optimlaity...
Loading all itr-world data (40) for alg (asy-sonata)...
Aggregating data...
Computing optimlaity...
Loading all time-world data (40) for alg (asy-sonata)...
Distilling data...
Aggregating data...
Computing optimlaity...
Loading all itr-world data (40) for alg (asy-sonata)...
Aggregating data...
Computing optimlaity...
Loading all time-w

In [128]:
for alg in lr_data_dict:
    print(alg, len(lr_data_dict[alg][40]['erdos-renyi']['obj_itr']['step-size:100.0']))

asy-sonata 34499
agp 32622
gp 35001
pd 35001
ep 35001


In [129]:
import bokeh.palettes as ptestbkh
ptestbkh.__palettes__

['Accent3',
 'Accent4',
 'Accent5',
 'Accent6',
 'Accent7',
 'Accent8',
 'Blues3',
 'Blues4',
 'Blues5',
 'Blues6',
 'Blues7',
 'Blues8',
 'Blues9',
 'Blues256',
 'BrBG3',
 'BrBG4',
 'BrBG5',
 'BrBG6',
 'BrBG7',
 'BrBG8',
 'BrBG9',
 'BrBG10',
 'BrBG11',
 'BuGn3',
 'BuGn4',
 'BuGn5',
 'BuGn6',
 'BuGn7',
 'BuGn8',
 'BuGn9',
 'BuPu3',
 'BuPu4',
 'BuPu5',
 'BuPu6',
 'BuPu7',
 'BuPu8',
 'BuPu9',
 'Category10_3',
 'Category10_4',
 'Category10_5',
 'Category10_6',
 'Category10_7',
 'Category10_8',
 'Category10_9',
 'Category10_10',
 'Category20_3',
 'Category20_4',
 'Category20_5',
 'Category20_6',
 'Category20_7',
 'Category20_8',
 'Category20_9',
 'Category20_10',
 'Category20_11',
 'Category20_12',
 'Category20_13',
 'Category20_14',
 'Category20_15',
 'Category20_16',
 'Category20_17',
 'Category20_18',
 'Category20_19',
 'Category20_20',
 'Category20b3',
 'Category20b4',
 'Category20b5',
 'Category20b6',
 'Category20b7',
 'Category20b8',
 'Category20b9',
 'Category20b10',
 'Category20b11

In [159]:
print('Plotting learning-rate sweep')
print('------------------------------------------')

num_colors = 5
if debug:
    from bokeh.palettes import Colorblind8 as palette
    colors = itertools.cycle(palette)
    g_colors = (c[1] for c in zip(range(num_colors), colors))    
    p = figure(plot_width=800, plot_height=400,
               title='Residual error after 35000 iterations swept against the step-size',
               y_axis_type='log',
               x_axis_type='log')
    p.yaxis.axis_label = 'Residual Error'
    p.xaxis.axis_label = 'Step-Size'
    p.y_range.end = 1e4
    axes = [_, _]
else:
    g_colors = itertools.cycle(colors)
#     g_colors = (c[1] for c in zip(range(num_colors), g_colors))
    _, axes = plt.subplots(nrows=1, ncols=2, figsize=figsize_double, sharey=False)
        
plot_algs_list = ['agp', 'gp', 'pd', 'ep', 'asy-sonata']
plot_graphs_list = ['erdos-renyi', 'ring']
plot_worlds_list = [40]

lr_list = [list(tag.values())[0] for tag in tag_list]
for alg, clr in zip(plot_algs_list, g_colors):
    print(alg, clr)
    for ax, gph in zip(axes, plot_graphs_list):

        # Compute residual error
        yl_lr_list = []
        for key in lr_data_dict[alg][plot_worlds_list[0]][gph]['obj_itr']:
            try:
                if alg == 'agp' or alg == 'asy-sonata':
                    min_yl = min(min(lr_data_dict[alg][plot_worlds_list[0]][gph]['obj_time'][key]), 1e21)
                else:
                    min_yl = min(lr_data_dict[alg][plot_worlds_list[0]][gph]['obj_time'][key][-1], 1e21)
                    if isnan(min_yl):
                        min_yl = 1e21
            except Exception as e:
                min_yl = 1e21
            yl_lr_list.append(min_yl)
            try:
                print('(%s) residual(%s) num-updates/time: %s/%s' %
                      (alg, min_yl,
                       lr_data_dict[alg][plot_worlds_list[0]][gph]['num_updates'][key][0],
                       lr_data_dict[alg][plot_worlds_list[0]][gph]['time'][key][-1]))
            except:
                print('(%s) residual(%s)' % (alg, min_yl))

        # Plot residual error
        legend = '(%s) %s' % (gph, legend_alg[alg])
        if debug:
            line_dash = 'dashed'
            if 'gp' in alg:
                p.circle(lr_list, yl_lr_list, legend_label=legend, size=22, color=clr, alpha=.4)
            else:
                p.square(lr_list, yl_lr_list, legend_label=legend, size=12, color=clr, alpha=1)
            p.line(lr_list, yl_lr_list, legend_label=legend, color=clr, line_dash=line_dash)
        else:
            try:
                if 'gp' in alg:
                    markersize, alpha, marker = 25, 0.7, 'o'
                else:
                    markersize, alpha, marker = 15, 0.7, 's'
                ax.loglog(lr_list[:len(yl_lr_list)], yl_lr_list,
                          color=clr,
                          linestyle=linestyles[1],
                          label=legend,
                          linewidth=1.*linewidth,
                          marker=marker,
                          markersize=markersize,
                          alpha=alpha)
            except Exception as e:
                print('%s/%s error: %s' % (alg, gph, e))
                
if debug:
    # p.legend.orientation = 'horizontal'
    p.legend.location = 'top_left'
    p.output_backend = 'svg'
    show(p)
else:
    for ax in axes:
        ax.set_title(r'Residual Error Step-Size Sweep ($K = 35,000$ itr.)', fontsize=fontsize)
        ax.tick_params(axis='both', labelsize=fontsize)
        ax.legend(loc='best', fontsize=fontsize)
        ax.set_ylim(bottom=1e-15, top=1e22)
        ax.set_xlabel(r'Step-Size $\alpha$', fontsize=fontsize)
    axes[0].set_ylabel(r'$\Vert \mathbf{\overline{x}}[k] - \mathbf{x^{\star}} \Vert_2$', fontsize=fontsize)

    fig_newton = './assets/residual_stepsize_sweep_all.pdf'
    plt.savefig(fig_newton, bbox_inches='tight', pad_inches=0)
    plt.show()
    plt.close('all')

print('Done.')

Plotting learning-rate sweep
------------------------------------------
agp #1f77b4
(agp) residual(6.538316491500212) num-updates/time: 56341/1199.25
(agp) residual(3.9205033644750595) num-updates/time: 35395/1199.25
(agp) residual(0.08406294516690754) num-updates/time: 33551/1199.25
(agp) residual(4.456045194908325e-06) num-updates/time: 42219/1199.25
(agp) residual(6.936923983555813e-06) num-updates/time: 48970/1199.25
(agp) residual(4.2072963905331574e-06) num-updates/time: 33644/1199.25
(agp) residual(6.996524595854618) num-updates/time: 6452/1199.25
(agp) residual(3.234968190111086) num-updates/time: 47584/1199.25
(agp) residual(0.08346608193485137) num-updates/time: 33918/1199.25
(agp) residual(8.739013029264115e-06) num-updates/time: 69899/1199.25
(agp) residual(2.955192878734274e-05) num-updates/time: 17333/1199.25
(agp) residual(3.936020750160619e-05) num-updates/time: 22491/1199.25
gp #2ca02c
(gp) residual(6.752758389325979) num-updates/time: 35001/1199.25
(gp) residual(5.004

In [124]:
print('Plotting data vs. iterations')


num_colors = 8
if debug:
    from bokeh.palettes import Colorblind8 as palette
    colors = itertools.cycle(palette)
    g_colors = (c[1] for c in zip(range(num_colors), colors))    
    p = figure(plot_width=800, plot_height=400,
               title='Residual error after 35000 iterations swept against the step-size',
               y_axis_type='log',
               x_axis_type='log')
    p.yaxis.axis_label = 'Residual Error'
    p.xaxis.axis_label = 'Step-Size'
    p.y_range.end = 1e4
    axes = [_, _]
else:
    g_colors = itertools.cycle(colors)
    g_colors = (c[1] for c in zip(range(num_colors), g_colors))
    _, axes = plt.subplots(nrows=1, ncols=2, figsize=figsize_double, sharey=False)

plot_algs_list = ['asy-sonata', 'agp', 'pd', 'ep', 'gp']
plot_graphs_list = ['erdos-renyi']
plot_worlds_list = [40]
plot_lr = {
    'agp': 'step-size:100.0',
    'asy-sonata': 'step-size:100.0',
    'ep': 'step-size:100.0',
    'gp': 'step-size:100.0',
    'pd': 'step-size:100.0'
}

p = figure(title='Average error sequence',
           y_axis_type='log')
p.yaxis.axis_label = 'global loss'
p.xaxis.axis_label = 'iterations'

for alg in plot_algs_list:
    for ws in plot_worlds_list:
        for gph in plot_graphs_list:
            yl = lr_data_dict[alg][ws][gph]['obj_itr'][plot_lr[alg]]
            xl = np.arange(len(yl))
            p.line(xl, yl, legend_label='%s:%s' % (legend_alg[alg], gph), color=next(g_colors), line_width=4)
show(p)
print('Done.')

Plotting data vs. iterations


Done.
